# مبدل فایل سرفصل درس 

این برنامه تبدیل فایل سرفصل درسها به صورت مارک داون که قابل درج در گیت‌هاب باشد را انجام می‌دهد.

فرض بر این هست که سرفصل همه درسها در یک فایل مارک‌داون است، این برنامه بخشهای مختلف هر درس از قبیل،‌عنوان، تعداد واحد، نوع درسها و ... را استخراج کرده و مشخصات ار درس را در یک فایل مارک‌داون قرار می‌دهد.
سپس از میک‌داکس برای ایجاد صفحات مربوطه استفاده می‌شود

https://products.aspose.app/total/conversion

https://products.groupdocs.app/conversion/docx-to-md

## M. Amintoosi

In [13]:
# !pip install markdown-strings


In [14]:
import io
import re
import markdown_strings
import pandas as pd


In [15]:
def extract_elements(input_str):
    start_idx = input_str.find('|',1)
    start_idx = input_str.find('|',start_idx) # Move to next |
    end_idx = input_str.find('|',start_idx+1)
    prreq = input_str[start_idx+1:end_idx]
    prreq = re.sub('\*', '', prreq)
    prreq = prreq.strip()
    if prreq == '':
        prreq = 'ندارد'
    # print(prreq)

    # نوع درس
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    c_cat = input_str[start_idx+1:end_idx]
    c_cat = re.sub('\*', '', c_cat)
    if 'n' in c_cat  or '¢' in c_cat:
        c_cat = c_cat.replace('n','').strip()
        c_cat = c_cat.replace('¢','').strip()
    elif 'حل تمرین' in c_cat:
        c_cat = re.sub('حل تمرین','',c_cat)
    else:
        c_cat = ''
    # print(c_cat)

    # نوع واحد
    start_idx = end_idx
    end_idx = input_str.find('|',start_idx+1)
    unit_type = input_str[start_idx+1:end_idx]
    unit_type = re.sub('\*', '', unit_type)
    if 'n' in unit_type or '¢' in unit_type:
        unit_type = unit_type.replace('n','').strip()
        unit_type = unit_type.replace('¢','').strip()
    else:
        unit_type = ''
    # print(unit_type)
    return prreq, c_cat, unit_type

In [16]:
# Load the file into file_content
file_content = [line for line in open('../input/CS-Chart.md')]
col_names = ['fa_title','en_title','en_file_name','c_cat','complementary','prreq','coreq',\
                           'c_unit','unit_type','c_hours','c_ex','c_goal1','c_goal2','sar_fasl','c_refs']
df = pd.DataFrame(columns=col_names)

fa_col_names = ['عنوان درس به فارسی:', 'عنوان درس به انگلیسی:', 'نوع درس:', 'آموزش تکمیلی:','پیش‌نیاز:','هم‌نیاز',\
                 'تعداد واحد:', 'نوع واحد:', 'تعداد ساعت:','حل تمرین:', 'هدف کلی:', 'اهداف ویژه:',\
                      'سر فصل یا رئوس مطالب','فهرست منابع پیشنهادی']
# df.set_index('fa_title',inplace = True)
line_no = 0
while line_no < len(file_content):
    line = file_content[line_no]
    pattern = r"\| --- \|"
    result = re.sub(pattern, "", line)
    # pattern = r"\|"
    # result = re.sub(pattern, "", result)
    srch_str = "عنوان درس به فارسی:"
    # print(result.find(srch_str))
    if result.find(srch_str) != -1:
        result = re.sub("<a.*?/a>",'',result)
        start_idx = result.find('<h3>') + 4
        end_idx = result.rfind('</h3>')
        fa_title = result[start_idx:end_idx]
        fa_title = re.sub('\*', '', fa_title)
        fa_title = fa_title.strip()
        line_no += 1
        print(fa_title)
        continue

    srch_str = "عنوان درس به انگلیسی:"
    # print(result , result.find(srch_str))
    if result.find(srch_str) != -1:
        start_idx = result.find('|',1)
        end_idx = result.find('|',start_idx+1)
        en_title = result[start_idx+1:end_idx]
        en_title = re.sub('\*', '', en_title)
        en_title = re.sub('<.*?>', '', en_title)
        en_title = en_title.strip()
        en_file_name = re.sub('\&', 'and', en_title)
        en_file_name = re.sub('\n', '', en_file_name)
        en_file_name = re.sub(' ', '-', en_file_name)
        # print(en_title)

        # پیش نیاز
        line_no += 1
        line = file_content[line_no]
        prreq, c_cat, unit_type = extract_elements(line)
        # print(prreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        coreq, c_cat_, unit_type_ = extract_elements(line)
        # if c_cat_ != '':
        #     print('\n', c_cat_, '\n')
        c_cat, unit_type = c_cat+c_cat_,  unit_type+unit_type_
        # print(coreq, c_cat, unit_type)

        line_no += 1
        line = file_content[line_no]
        c_unit, c_ex, c_cat_ = extract_elements(line)
        if unit_type == '':
            unit_type = "نظری-عملی"
        c_cat = c_cat+c_cat_
        # print(c_unit, c_ex, c_cat)

        line_no += 1
        line = file_content[line_no]
        c_hours, _, _ = extract_elements(line)
        print(c_hours, '\n')
       
        while True:
            line_no += 1
            if line_no >= len(file_content):
                break
            line = file_content[line_no]
            if line.find(fa_col_names[-2]) != -1: #سرفصل
                break
        if line_no >= len(file_content):
                break

        # continue
        # خواندن سرفصل
        sar_fasl =[]
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find(fa_col_names[-1]) == -1: #refs
            if line.strip() != '':
                sar_fasl.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]

        if line_no >= len(file_content):
            break    

        refs = []
        line_no += 1
        line = file_content[line_no]
        while line_no<len(file_content) and line.find("عنوان درس به فارسی:") == -1:
            if line.strip() != '':
                refs.append(line)
            line_no += 1
            if line_no>= len(file_content):
                break
            line = file_content[line_no]
            # print(line_no)

        if line.find("عنوان درس به فارسی:") != -1:
            line_no -= 1
        this_course ={'fa_title':fa_title, 'en_title':en_title, 'en_file_name':en_file_name,'c_cat':c_cat,'prreq':prreq,'coreq':coreq,\
                           'c_unit':c_unit,'unit_type':unit_type,'c_hours':c_hours,'c_ex':c_ex,\
                            'sar_fasl':sar_fasl,'c_refs':refs}
        df = df.append(this_course,ignore_index=True)
    line_no += 1
    # print('')
    
# df

ریاضی عمومی ۱
64 

ریاضی عمومی ۲
64 

معادلات دیفرانسیل
48 

مبانی کامپیوتر و برنامه‌نویسی
48 

کارگاه کامپیوتر ۱
32 

مبانی علوم ریاضی
48 

آمار و احتمال ۱
48 

نرم‌افزارهای آماری و تحلیل داده‌ها
32 

برنامه‌نویسی پیشرفته
48 

برنامه‌نویسی پایتون
48 

ساختمان داده‌ها
48 

طراحی و تحلیل الگوریتم‌ها
48 

کارگاه کامپیوتر ۲
32 

پایگاه داده‌ها
48 

اصول سیستم‌های کامپیوتری
48 

مبانی ترکیبیات
48 

مبانی منطق و نظریه مجموعه‌ها
48 

بهینه‌سازی گسسته
48 

مبانی نظریه محاسبه
48 

مبانی محاسبات علمی
48 

مبانی آنالیز عددی
48 

مبانی آنالیز ریاضی
48 

تحلیل آماری داده‌ها
48 

سری‌های زمانی کاربردی
48 

یادگیری ماشین مقدماتی
48 

داده‌کاوی مقدماتی
48 

تحقیق در عملیات
48 



بهینه‌سازی غیرخطی
48 

طراحی و توسعه کسب و کارهای نوپا
48 

مبانی اقتصاد دیجیتال
32 

کارآموزی
192 

هوش مصنوعی
48 

آمار و احتمال ۲
48 

آمار محاسباتی
48 

روش‌های آماری
48 

نظریه محاسبه
48 

منطق برای علوم کامپیوتر
48 

نظریه مقدماتی کدگذاری
48 

رمزنگاری
48 

برنامه‌نویسی امن
48 

معناشناسی عملیاتی برنامه‌نویسی
48 

آشنایی با نظریه بازی‌ها
48 

الگوریتم‌های تصادفی
48 

هندسه محاسباتی
48 

رگرسیون ۱
48 

هوش محاسباتی
48 

هوش تجاری مقدماتی
48 

آشنایی با یادگیری عمیق
48 

تجارت الکترونیک
32 

مدلسازی ریاضی
48 

آزمایشگاه ریاضی
32 

ریاضیات فازی
48 

نظریه گراف و کاربردها
48 

مبانی جبر
48 

مبانی آنالیز فوریه و موجک ها
48 

پردازش تصویر مقدماتی
48 

برنامه‌نویسی موبایل
48 

برنامه‌نویسی وب
48 

شبکه‌های کامپیوتری
48 

شبیه‌سازی کامپیوتری
48 

گرافیک کامپیوتری
48 

تحلیل شبکه‌های اجتماعی
48 

سیستم‌های عامل
48 

اصول طراحی نرم‌افزار
48 

کامپایلر
48 

مباحثی در علوم کامپیوتر ۱
48 

مباحثی در علوم کامپیوتر ۲
48 

مبانی بیوانفورماتیک
48 

پروژه کارشناسی
96 

آنالیز عددی
48 

آشنایی با 

In [17]:
df.to_excel('output/lessons.xlsx')

In [18]:
def table_from_rows(table_list):
    """Return a formatted table, using each list as the list. The specifics are
    the same as those for the table function.

    >>> table_from_rows([["1","2","3"],["4","5","6"],["7","8","9"]])
    '| 1 | 2 | 3 |\\n| --- | --- | --- |\\n| 4 | 5 | 6 |\\n| 7 | 8 | 9 |'
    """
    # transpose the list
    number_of_rows = len(table_list[0])
    transposed = []
    for column_number in range(0, number_of_rows):
        column_list = [row[column_number] for row in table_list]
        transposed.append(column_list)

    return markdown_strings.table(transposed)


شناسایی دسته هر درس، مشخص کردن نام فایل درس

In [19]:
for index, row in df.iterrows():
    # print(row['fa_title'], row['en_title'])
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
    # print(r_path)
    lesson_name_en = row['en_title'].strip()
    lesson_name_en = re.sub('\&', 'and', lesson_name_en)
    lesson_name_en = re.sub(' ', '-', lesson_name_en)
    lesson_file_name = r_path+'/'+lesson_name_en+'.md'
    row['en_file_name'] = lesson_file_name
    
# df.head()   

In [20]:
df

,fa_title,en_title,en_file_name,c_cat,complementary,prreq,coreq,c_unit,unit_type,c_hours,c_ex,c_goal1,c_goal2,sar_fasl,c_refs
0,ریاضی عمومی ۱,Calculus I,base/Calculus-I.md,پایه,NaN,ندارد,ندارد,4,نظری,64,دارد,NaN,NaN,[مختصات دکارتی، مختصات قطبی، اعداد مختلط، جمع ...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
1,ریاضی عمومی ۲,Calculus II,base/Calculus-II.md,پایه,NaN,ریاضی عمومی ۱,ندارد,4,نظری,64,دارد,NaN,NaN,[سیستم مختصات قطبی، نمودارهای قطبی دنباله، سری...,"[-\n, - - استوارت، ج. (1395). حساب دیفرانسیل و..."
2,معادلات دیفرانسیل,Differential Equations,base/Differential-Equations.md,پایه,NaN,ریاضی عمومی ۱,ندارد,3,نظری,48,دارد,NaN,NaN,[مفاهیم و تعاریف اولیه .معادلات جداپذیر و معاد...,[- - ظهوری زنگنه، ح. (مترجم). (1394). معادلات ...
3,مبانی کامپیوتر و برنامه‌نویسی,Fundamentals of Computer Programming,base/Fundamentals-of-Computer-Programming.md,پایه,NaN,ندارد,ندارد,3,نظری,48,دارد,NaN,NaN,[اهداف درس با هر زبان برنامه‌نویسی سطح بالا مح...,"[-\n, - - Matthes, E. (2019). Python Crash Cou..."
4,کارگاه کامپیوتر ۱,Computer Workshop I,base/Computer-Workshop-I.md,پایه,NaN,ندارد,مبانی کامپیوتر و برنامه‌نویسی,1,عملی,32,ندارد,NaN,NaN,[مطالب اصلی مورد بحث در اختیار گروه و استاد در...,[در اختیار مدرس\n]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,رایانش چند‌هسته‌ای,Multicore Computing,elective/Multicore-Computing.md,تخصصی اختیاری,NaN,برنامه‌نویسی پیشرفته، اصول سیستم‌های کامپیوتری,ندارد,3,نظری,48,دارد,NaN,NaN,[معرفی معماری سیستم‌های چندهسته‌ای و مدل‌های ب...,"[-\n, - - Patterson, D. A., & Hennessy, J. L. ..."
74,سیگنال‌ها و سیستم‌ها,Signals and Systems,elective/Signals-and-Systems.md,تخصصی اختیاری,NaN,معادلات دیفرانسیل,ندارد,3,نظری,48,دارد,NaN,NaN,[--مفاهیم اولیه، سیگنال های زمان پیوسته و گسست...,"[-\n, - - Oppenheim, A. V., Willsky, A. S., & ..."
75,مدیریت پروژه‌های فناوری اطلاعات,Information Technology Project Management,elective/Information-Technology-Project-Manage...,تخصصی اختیاری,NaN,ندارد,ندارد,3,نظری,48,ندارد,NaN,NaN,[مقدمات، اهداف، برنامه‌ی درسی و توافق بر الگوی...,"[-\n, - - Marchewka, J. T. (2014). Information..."
76,جبر خطی,Linear Algebra,elective/Linear-Algebra.md,تخصصی اختیاری,NaN,مبانی محاسبات علمی,ندارد,3,نظری,48,دارد,NaN,NaN,[**مروری بربردارها و ماتریسها:**جمع وتفریق و ض...,[- - داتا، ب.، (۱۳۹۵). جبر خطی عددی و کاربردها...


In [21]:
def prreq_links(prreq_str, path_mod = ''):
# تابعی برای استخراج نام هر درس از لیست درسهای پیش‌نیازی و درج لینک فایل مربوطه
    # prreq_str = "مبانی آنالیز ریاضی، مبانی ماتریس‌ها و جبر خطی "
    p_strs = prreq_str.split('،')
    p_list = []
    for p in p_strs:
        x = df[df['fa_title'].str.contains(p.strip())][['fa_title','en_file_name']]
        p_str = p
        if not x.empty:
            fa_title, en_file_name = x['fa_title'].values[0], x['en_file_name'].values[0]
            p_str = '['+fa_title+']('+path_mod+en_file_name+')'
        p_list.append(p_str)
    return ' ،'.join(p_list)


In [22]:
# prreq_str = "مبانی آنالیز ریاضی، مبانی ماتریس‌ها و جبر خطی ، مبانی آنالیز ریاضی، ریاضی عمومی ۱"
# print(prreq_links(prreq_str))


ایجاد فایل برای تک تک درسها با فرمت راست به چپ

In [23]:
# for index, row in df.iterrows():
#     # # print(row['fa_title'], row['en_title'])
#     # if 'پایه' in  row['c_cat']:
#     #     r_path = 'base'
#     # elif 'الزامی' in row['c_cat']:
#     #     r_path = 'mandatory'
#     # elif 'اختیاری' in row['c_cat']:
#     #     r_path = 'elective'
#     # # print(r_path)
#     # lesson_name_en = row['en_title'].strip()
#     # lesson_name_en = re.sub('\&', 'and', lesson_name_en)
#     # lesson_name_en = re.sub(' ', '-', lesson_name_en)
#     # lesson_file_name = r_path+'/'+lesson_name_en+'.md'
#     lesson_file_name = row['en_file_name']
#     lesson_writer = open(lesson_file_name, 'w')
#     md_str = markdown_strings.header(row['fa_title']+'\n', 1)
#     lesson_writer.write(md_str)
#     md_str = markdown_strings.header(row['en_title']+'\n', 2)
#     lesson_writer.write(md_str)
#     lesson_writer.write(markdown_strings.horizontal_rule())
#     lesson_writer.write('\n')
#     lesson_writer.write(table_from_rows([\
#         ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"],\
#         ["پیش‌نیاز:",prreq_links(row['prreq'],path_mod = '../'),"گروه درس:",row['c_cat']],\
#         ["هم‌نیاز:",row['coreq'],"نوع درس:",row['unit_type']],\
#         ["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']],\
#         ["حل تمرین:",row['c_ex'],"",""]\
#         ]))
#     lesson_writer.write('\n\n')
#     lesson_writer.write('**سر فصل یا رئوس مطالب**\n\n')
#     lesson_writer.write("\n".join(row['sar_fasl']))
#     # for line in row['sar_fasl']:
#     #     lesson_writer.write(f"{line}\n")
#     lesson_writer.write('\n**فهرست منابع پیشنهادی**\n\n')
#     lesson_writer.write("\n".join(row['c_refs']))
#     lesson_writer.close()
#     # if index > 15:
#     #     break

ایجاد فایل برای تک تک درسها با فرمت چپ به راست

In [24]:
ref_file_name = 'output/refs.txt'
ref_writer = open(ref_file_name, 'w')

for index, row in df.iterrows():
    lesson_file_name = row['en_file_name']
    lesson_writer = open(lesson_file_name, 'w')
    md_str = markdown_strings.header(row['fa_title']+'\n', 1)
    lesson_writer.write(md_str)
    md_str = markdown_strings.header(row['en_title']+'\n', 2)
    lesson_writer.write(md_str)
    lesson_writer.write(markdown_strings.horizontal_rule())
    lesson_writer.write('\n')
    lesson_writer.write(table_from_rows([\
        ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"][::-1],\
        ["پیش‌نیاز:",prreq_links(row['prreq'],path_mod = '../'),"گروه درس:",row['c_cat']][::-1],\
        ["هم‌نیاز:",row['coreq'],"نوع درس:",row['unit_type']][::-1],\
        ["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']][::-1],\
        ["حل تمرین:",row['c_ex'],"",""][::-1]\
        ]))
    lesson_writer.write('\n\n')
    lesson_writer.write('**سر فصل یا رئوس مطالب**\n\n')
    lesson_writer.write("\n".join(row['sar_fasl']))
    # for line in row['sar_fasl']:
    #     lesson_writer.write(f"{line}\n")
    lesson_writer.write('\n**فهرست منابع پیشنهادی**\n\n')
    lesson_writer.write("\n".join(row['c_refs']))
    lesson_writer.close()

    ref_writer.write("\n\n"+row['fa_title']+"\n")
    refs = [re.sub('\*', '', rf) for rf in row['c_refs']]
    print(refs)
    ref_writer.write("".join(refs))

ref_writer.close()


['-\n', '- - استوارت، ج. (1395). حساب دیفرانسیل و انتگرال (ج. حمیدی، مترجم). تهران: مؤسسه آموزشی و نشریاتی مدرسه.\n', '- - شهشهانی، س. (1396). حساب دیفرانسیل و انتگرال (ویراست 2). تهران: انتشارات فاطمی.\n', '- - Stewart, J. (2016). Calculus (8th ed.). Cengage Learning.\n', '- - Adams, R. A., & Essex, C. (2018). Calculus: A complete course (9th ed.). Pearson.\n', '- - Finney, T., Demana, F., Waits, B. K., Kennedy, D., & Rodgers, J. (2014). Calculus: Graphical, numerical, algebraic (4th ed.). Pearson.\n']
['-\n', '- - استوارت، ج. (1395). حساب دیفرانسیل و انتگرال (ج. حمیدی، مترجم). تهران: مؤسسه آموزشی و نشریاتی مدرسه.\n', '- - شهشهانی، س. (1396). حساب دیفرانسیل و انتگرال (ویراست 2). تهران: انتشارات فاطمی.\n', '- - Stewart, J. (2016). Calculus (8th ed.). Cengage Learning.\n', '- - Adams, R. A., & Essex, C. (2018). Calculus: A complete course (9th ed.). Pearson.\n', '- - Finney, T., Demana, F., Waits, B. K., Kennedy, D., & Rodgers, J. (2014). Calculus: Graphical, numerical, algebraic (4th e

In [25]:
# print(["تعداد واحد:",row['c_unit'],"تعداد ساعت:",row['c_hours']].reverse())
row_el = ["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"]
print(list(row_el))
["نام درس:",row['fa_title'],"مقطع:", "کارشناسی"][::-1]
# row['fa_title']

['نام درس:', 'بازی\u200cسازی و بازی\u200cانگاری', 'مقطع:', 'کارشناسی']


['کارشناسی', 'مقطع:', 'بازی\u200cسازی و بازی\u200cانگاری', 'نام درس:']

ایجاد فایل yml

In [26]:
import locale
locale.setlocale(locale.LC_ALL, 'fa_IR.utf8')
# نام درسها رو بر اساس ترتیب صحیح الفبای فارسی مرتب می‌کنیم
# اما نام دسته را بر همان اساس ترتیب نادرست که پ آخره و در ترتیب معکوس، میشه اول
df_temp = df.sort_values(by = 'fa_title', key = lambda col: col.map(lambda x: locale.strxfrm(x)))
df_sorted = df_temp.sort_values(by = 'c_cat', ascending=[False], kind = 'mergesort')
# df_sorted = df.sort_values(by=['c_cat', 'fa_title'], ascending=[False,True])
# df_sorted

In [27]:
# df['c_cat'].head()
# df_sorted
# locale.strxfrm('پایه') < locale.strxfrm('ان')

In [28]:
yml_file_name = '../mkdocs.yml'
yml_writer = open(yml_file_name, 'w')
yml_str = "site_name: علوم کامپیوتر فردوسی \n\
nav:\n\
  - صفحه اصلی: index.md\n\
  - دروس پایه:\n\
"
yml_writer.write(yml_str)
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        yml_str = "  - دروس الزامی:"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        yml_str = "  - دروس اختیاری:"

    if curr_cat != prev_cat:
        yml_writer.write(yml_str)
        yml_writer.write('\n')
    prev_cat = curr_cat
    # print(r_path)
    # lesson_file_name = r_path+'/'+row['en_file_name']+'.md'
    # print(lesson_file_name)
    # yml_str = '    - "'+row['fa_title']+'": '+lesson_file_name+'\n'
    yml_str = '    - "'+row['fa_title']+'": '+row['en_file_name']+'\n'
    yml_writer.write(yml_str)

    # if index > 80:
    #     break

# yml_writer.write('\n\ntheme: jinks_en')
yml_writer.write('\n\ntheme:\n')
yml_writer.write('    name: mkdocs\n')
yml_writer.write('    locale: fa\n')
yml_writer.close()


تبدیل سرفصلها با سایت زیر
https://products.aspose.app/words/conversion/word-to-md

تبدیل ایندکس با سایت زیر
https://alldocs.app/convert-word-docx-to-markdown


In [29]:
# !pip install mkdocs-jinks

!mkdocs serve
خطا داشت!

And then: mkdocs build

Copy site folder contents to cs-um/docs

In [30]:
# !mkdocs build

## Generate README.md, SUMMARY.md for gitbook

in https://cs-um.gitbook.io/curr/

In [31]:
rdme_file_name = '../README.md'
rdme_writer = open(rdme_file_name, 'w')
rdme_str = '# خوش آمدید \n\
{% hint style="info" %}\n\
**درباره این سایت:** این سایت صفحه غیررسمی گروه علوم کامپیوتر و مرتبط با برنامه درسی دوره کارشناسی علوم کامپیوتر فردوسی می‌باشد\n\
سایت، غیر رسمی و در حال راه‌اندازی است و مطالب آن قابلیت استناد ندارند.\n\
{% endhint %}\n\n\
<details>\n\
<summary>دروس پایه</summary>\n\n\
'
rdme_writer.write(rdme_str)
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        rdme_str = "دروس الزامی"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        rdme_str = "دروس اختیاری"

    if curr_cat != prev_cat:
        rdme_writer.write('\n</details>\n')
        rdme_writer.write('\n<details>\n')
        rdme_writer.write('\n<summary>')
        rdme_writer.write(rdme_str)
        rdme_writer.write('</summary>\n')
        rdme_writer.write('\n')
    prev_cat = curr_cat
    # print(r_path)
    # lesson_file_name = 'https://cs-um.gitbook.io/curr/docs/'+r_path+'/'+row['en_file_name']
    # lesson_file_name = 'docs/'+r_path+'/'+row['en_file_name']+'.md'
    lesson_file_name = 'docs/'+r_path+'/'+row['en_file_name']
    # print(lesson_file_name)
    rdme_str = '* ['+row['fa_title']+']('+row['en_file_name']+')\n'
    rdme_writer.write(rdme_str)

    # if index > 80:
    #     break

rdme_writer.write('\n</details>\n')
rdme_writer.close()


In [32]:
# For Gitbook
rdme_file_name = '../Tables.md'
rdme_writer = open(rdme_file_name, 'w')
rdme_str = '\
---\n\
dir: rtl\n\
---\n\n\
# جداول دروس پایه، تخصصی و اختیاری\n\n'
rdme_writer.write(rdme_str)
rdme_writer.write('\n## '+'دروس پایه\n\n')
rdme_writer.write(table_from_rows([["تعداد واحد","پیش‌نیاز","نام درس"]]))
num_units = 0
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        rdme_str = "دروس الزامی"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        rdme_str = "دروس اختیاری"
    if curr_cat != prev_cat:
        rdme_writer.write('\n |'+ num_units.__str__()+'|'+'مجموع تعداد واحد'+'|\n')
        num_units = 0
        rdme_writer.write('\n## '+rdme_str)
        rdme_writer.write('\n\n')
        rdme_writer.write(table_from_rows([["تعداد واحد","پیش‌نیاز","نام درس"]]))

    prev_cat = curr_cat
    num_units += int(row['c_unit'])
    rdme_writer.write('\n|'+row['c_unit']+'|'+\
        prreq_links(row['prreq'],path_mod = 'docs/') +'|'+\
            '['+row['fa_title']+'](docs/'+row['en_file_name']+') | ')
    # if index > 80:
    #     break
rdme_writer.write('\n |'+ num_units.__str__()+'|'+'مجموع تعداد واحد'+'|\n')
rdme_writer.close()


In [33]:
rdme_file_name = '../SUMMARY.md'
rdme_writer = open(rdme_file_name, 'w')
rdme_str = '\
* [صفحه اصلی](README.md)\n\
* [مشخصات کلی](general-specifications.md)\n\
* [جداول دروس](Tables.md)\n\
## دروس پایه\n\
'
rdme_writer.write(rdme_str)
cat_changed = False
r_path = 'base'
prev_cat = r_path
for index, row in df_sorted.iterrows():
    if 'پایه' in  row['c_cat']:
        r_path = 'base'
        curr_cat = r_path
    elif 'الزامی' in row['c_cat']:
        r_path = 'mandatory'
        curr_cat = r_path
        rdme_str = "دروس الزامی"
    elif 'اختیاری' in row['c_cat']:
        r_path = 'elective'
        curr_cat = r_path
        rdme_str = "دروس اختیاری"

    if curr_cat != prev_cat:
        rdme_writer.write('## ')
        rdme_writer.write(rdme_str)
        rdme_writer.write('\n')
    prev_cat = curr_cat
    # lesson_file_name = 'docs/'+r_path+'/'+row['en_file_name']+'.md'
    lesson_file_name = 'docs/'+row['en_file_name']
    rdme_str = '* ['+row['fa_title']+']('+lesson_file_name+')\n'
    rdme_writer.write(rdme_str)
rdme_writer.close()
